In [ ]:
import pandas as pd
import numpy as np
import pickle

path = '../preprocess/'
path2 = '../raw/'
train_preprocess_1 = pd.read_csv(path+'train_preprocess_1.csv', index_col=[0])
test1_preprocess_1 = pd.read_csv(path+'test1_preprocess_1.csv', index_col=[0])
test2_preprocess_1 = pd.read_csv(path+'test2_preprocess_1.csv', index_col=[0])
activity = pd.read_csv(path2 + 'train_activity.csv')
label = pd.read_csv(path2 + 'train_label.csv')

def lstm(test_set = test1_preprocess_1, predictY = 'survival_time'):

    import tensorflow as tf
    import numpy as np
    import matplotlib
    import pandas as pd
    import os

    tf.set_random_seed(777)  # reproducibility

    def MinMaxScaler(data):

        numerator = data - np.min(data, 0)
        denominator = np.max(data, 0) - np.min(data, 0)
        # noise term prevents the zero division
        return numerator / (denominator + 1e-7)

    def normalize(df):
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        scaler.fit(df)
        df_scaled = scaler.transform(df)
        return df_scaled

    # train Parameters
    seq_length = 28 # 한 유저의 28일간 데이터를 사용함.
    data_dim = 33 # acc_id, day를 제외하고 33개의 피쳐를 가짐
    hidden_dim = 10 # 사용자 임의
    output_dim = 1 # 출력노드는 1
    learning_rate = 0.01
    iterations = 2000 # 역전파 횟수
    
    print(1)

    # 데이터 로드
    df = train_preprocess_1
    unique = activity.acc_id.unique()
    acc_id_lst = sorted(list(unique)*28)
    day_lst = (list(range(1,29))*len(unique))
    full_acc_id_day = pd.DataFrame({'acc_id' : acc_id_lst,
                                    'day' : day_lst})

    
    new_label= label.sort_values('acc_id').reset_index(drop=True)
    full_label_result = full_acc_id_day.merge(new_label,on=['acc_id'], how = 'left')

    label_series =full_label_result[predictY]
    df[predictY] = label_series
    
    print(2)

    # train/test split
    # train_size = int(len(xy) * 0.7)
    xy = df.values[:,2:]
    train_set = xy
    test_set = test_set.values[:,2:]

    # Scale each
    train_set[:,:-1] = normalize(train_set[:,:-1])
    test_set = normalize(test_set)
    
    print(3)

    # build datasets
    def build_dataset(time_series, seq_length):
        dataX = []
        dataY = []
        for i in range(0, len(time_series) - seq_length + 1, seq_length):
            _x = time_series[i:i + seq_length, :-1]
            _y = time_series[i + seq_length - 1, [-1]]  # Next close price
    #         print(_x, "->", _y)
            dataX.append(_x)
            dataY.append(_y)
        return np.array(dataX), np.array(dataY)

    trainX, trainY = build_dataset(train_set, seq_length)
    
    print(4)

    def build_dataset_for_test(time_series, seq_length):
        dataX = []
    #     dataY = []
        for i in range(0, len(time_series) - seq_length + 1, seq_length):
            _x = time_series[i:i + seq_length]
    #         _y = time_series[i + seq_length - 1, [-1]]  # Next close price
    #         print(_x, "->", _y)
            dataX.append(_x)
    #         dataY.append(_y)
        return np.array(dataX)

    testX = build_dataset_for_test(test_set, seq_length)
    
    print(5)

    # input place holders
    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, 1])

    # build a LSTM network
    cell = tf.contrib.rnn.BasicLSTMCell(
        num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh) #reuse = tf.get_variable_scope().reuse
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(
        outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

    # cost/loss
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares

    # optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)
    
    print(6)
    


    loss_lst = []
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):
            _, step_loss = sess.run([train, loss], feed_dict={
                                    X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))
            loss_lst.append(step_loss)

        # Test step
        
        test_predict = sess.run(Y_pred, feed_dict={X: testX})
        
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.plot(loss_lst)
    

    del df[predictY]
    return test_predict


import tensorflow as tf
with tf.variable_scope("lstm",reuse=tf.AUTO_REUSE) as scope:
    test1_survival_time_predict = lstm(test1_preprocess_1, 'survival_time')
    
    test1_amount_spent_predict = lstm(test1_preprocess_1, 'amount_spent')
    test2_survival_time_predict = lstm(test2_preprocess_1, 'survival_time')
    test2_amount_spent_predict = lstm(test2_preprocess_1, 'amount_spent')

test1_label = pd.DataFrame({'acc_id' : test1_preprocess_1.acc_id.unique(),
              'survival_time' : test1_survival_time_predict.reshape(-1),
              'amount_spent' : test1_amount_spent_predict.reshape(-1)})

test2_label = pd.DataFrame({'acc_id' : test2_preprocess_1.acc_id.unique(),
              'survival_time' : test2_survival_time_predict.reshape(-1),
              'amount_spent' : test2_amount_spent_predict.reshape(-1)})

test1_label.to_csv('test1_label.csv', encoding = 'utf8')
test2_label.to_csv('test2_label.csv', encoding = 'utf8')